In [ ]:
!python -V
!pip list

In [1]:
!pip install -q -U git+https://github.com/fabienfrfr/tptt@main

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 91.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 

In [2]:
import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, DataCollatorForLanguageModeling, TrainerCallback
from datasets import load_dataset
import tptt

2025-05-24 10:41:54.872278: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748083315.312462      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748083315.427383      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Step 1: Load configuration and initialize the TPTT model
# Using a pretrained backbone (TinyLlama in this example)
config = tptt.TpttConfig(base_model_name="TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T")#, mag_weight=0.1, inject_liza=False)
model = tptt.TpttModel(config)

# Step 2: (Optional) Inject LoRA adapters for parameter-efficient fine-tuning
model.add_lora()

# Step 3: Load the tokenizer corresponding to the base model
tokenizer = AutoTokenizer.from_pretrained(config.base_tokenizer_name)
# Ensure the tokenizer has a padding token for batching
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token or "[PAD]"

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [4]:
# Step 4: Prepare the training dataset
# Here we use a small subset of the Alpaca dataset for demonstration purposes
raw_dataset = load_dataset("yahma/alpaca-cleaned")["train"].select(range(100))

def preprocess_fn(samples):
    """
    Tokenize the samples for causal language modeling.
    Concatenate instruction, input, and output as needed.
    """
    prompts = [
        f"{instr}\n{inp}" if inp else instr
        for instr, inp in zip(samples["instruction"], samples["input"])
    ]
    # Optionally, append output for supervised fine-tuning
    prompts = [f"{p}\n{out}" for p, out in zip(prompts, samples["output"])]
    tokens = tokenizer(
        prompts,
        truncation=True,
        max_length=256,
        padding="max_length",
        return_attention_mask=True,
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = raw_dataset.map(
    preprocess_fn, batched=True, remove_columns=raw_dataset.column_names
)

# Tokenize the dataset in batches and remove original columns
tokenized_dataset = raw_dataset.map(
    preprocess_fn, batched=True, remove_columns=raw_dataset.column_names)

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [5]:
# Step 5: Set up a data collator for dynamic padding during training
#data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length")
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)


# Step 6: Define HuggingFace TrainingArguments for reproducible training
training_args = TrainingArguments(
    output_dir="./tptt_output",
    per_device_train_batch_size=4, # per_device_train_batch_size * N GPU --> VRAM limit risk
    num_train_epochs=10,
    learning_rate=  5e-4, #1e-5, #2e-4, too brutal ?
    max_grad_norm=1.0, # gradiant clipping
    fp16=True, #fp16=True,  # Use mixed precision if supported by hardware
    logging_steps=5,
    save_strategy="epoch",
    report_to="tensorboard",
)

# Step 7: Initialize the HuggingFace Trainer
initial_weight=0.01,
final_weight=0.5,
transition_step=100,
liza_callback = tptt.AdjustMaGWeightCallback(
            model,
            initial_weight=initial_weight,
            final_weight=final_weight,
            transition_step=transition_step,)

# Trainer will automatically handle device placement (CPU/GPU)
trainer = Trainer(
    model=model,#.backbone,  # Use the underlying HF model for training
    args=training_args,
    #label_names=["labels"],  # (peft warning, but doesn't exist!)
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    processing_class=tokenizer,
    callbacks=[liza_callback],
    #callbacks=[DebugLossCallback()],
)

# batch = next(iter(trainer.get_train_dataloader()))
# print("Labels:", batch["labels"][0]) # Verify if special tokens (-100) for padding
# print(batch.keys()) # dict_keys(['input_ids', 'attention_mask', 'labels'])

# Step 8: Launch training
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
5,0.867800
10,0.753100
15,0.756700
20,0.635800
25,0.710600
30,0.590300
35,0.617400
40,0.565700
45,0.485400
50,0.519500


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

TrainOutput(global_step=130, training_loss=0.48073567335422224, metrics={'train_runtime': 348.3308, 'train_samples_per_second': 2.871, 'train_steps_per_second': 0.373, 'total_flos': 1592471322624000.0, 'train_loss': 0.48073567335422224, 'epoch': 10.0, 'mag_weight': 0.4951})

In [14]:
# Step 9: Prepare the model for inference
# Move model to the desired device (e.g., "cuda:0" or "cpu") for generation
device = 0 if torch.cuda.is_available() else -1
model.to(f"cuda:{device}" if device != -1 else "cpu")
#model.eval() ## fix : RuntimeError: The size of tensor a (32) must match the size of tensor b (20) at non-singleton dimension 1
model.train()

# Step 10: Build the inference pipeline with the correct device and tokenizer
pipe = tptt.TpttPipeline(model=model.backbone, tokenizer=tokenizer, device=device)

# Step 11: Generate text from a prompt
result = pipe("Once upon a time,", max_new_tokens=150)
print(result[0]["generated_text"])  # Print the generated text

Device set to use cuda:0


Once upon a time, there was a kingdom. It was a beautiful, majestic kingdom, with a rich history and a bright future. The people of this kingdom were free to do as they pleased, as long as they followed the rules. The kingdom was safe, secure


In [ ]:
# Step 12: Save the trained model and tokenizer for future use or deployment
model.save_pretrained("./my_tptt_model")
tokenizer.save_pretrained("./my_tptt_model")

#### DEBUG PARTS

In [7]:
model

TpttModel(
  (backbone): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): LlamaForCausalLM(
        (model): LlamaModel(
          (embed_tokens): Embedding(32000, 2048)
          (layers): ModuleList(
            (0-21): 22 x LlamaDecoderLayer(
              (self_attn): LiZAttention(
                (base_attn): LlamaAttention(
                  (q_proj): lora.Linear(
                    (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=2048, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=2048, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embe

In [ ]:
### DEBUG Callback
class DebugLossCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None and "loss" in logs:
            print(f"[DEBUG][Callback] Step {state.global_step} loss: {logs['loss']}")

batch = next(iter(trainer.get_train_dataloader()))
print("\n[DEBUG] Batch keys:", batch.keys())
print("[DEBUG] input_ids (first 10):", batch["input_ids"][0][:10])
print("[DEBUG] labels (first 10):", batch["labels"][0][:10])
print("[DEBUG] labels unique:", torch.unique(batch["labels"]))
print("[DEBUG] nb non-masked labels:", (batch["labels"] != -100).sum().item())
output = model(**batch)
print("[DEBUG] Forward output.loss:", output.loss)